# Qiskit serverless demonstration: Hamiltonian simulation template

### Convert notebook to a Qiskit Serverless program

Here we use `nbconvert` to convert the Jupyter notebook to a Python program (`.py` file).

In [1]:
from pathlib import Path
from nbconvert.exporters import ScriptExporter

notebook_filename = "hamsim_template.ipynb"
source_path = Path("./source_files/")
source_path.mkdir(exist_ok=True)
script, script_info = ScriptExporter().from_filename(notebook_filename)
script_name = script_info["metadata"]["name"]
script_file = source_path / (script_name + script_info["output_extension"])
with script_file.open("w") as f:
    f.write(script)
print(f"Notebook saved to {script_file}")

Notebook saved to source_files/hamsim_template.py


In [2]:
with script_file.open() as f:
    print("".join(f.readlines()[:25]), "\n[...]")

#!/usr/bin/env python
# coding: utf-8

# # Reducing the Trotter error of Hamiltonian dynamics with <span style="color:black">approximate quantum compilation</span>
# 
# In this notebook, you will learn how to use **Approximate Quantum Compilation with Tensor Networks (AQC-Tensor)**.
# 
# This notebook can be executed either directly or as a Qiskit Serverless program (see [accompanying notebook](serverless_demo.ipynb)).  In a few places it behaves differently based on how it is called (i.e., by branching on the value of `is_running_in_serverless()`).
# 
# Our overall goal is to simulate time evolution of the above model Hamiltonian.  We do so by Trotter evolution, which we split into two portions:
# 
# 1. An initial portion that is simulable with matrix product states (MPS).  We will "compile" this portion using AQC as presented in https://arxiv.org/abs/2301.08609.
# 2. A subsequent portion of the circuit that will be executed only on hardware.

# ### Set parameters for function templat

### Upload template to serverless

In [3]:
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction

title = script_file.stem

template = QiskitFunction(
    title=script_file.stem,
    entrypoint=script_file.name,
    working_dir=str(source_path),
    dependencies=[
        "qiskit-addon-utils~=0.1.0",
        "qiskit-addon-aqc-tensor[quimb-jax]~=0.1.2",
        "mergedeep==1.3.4",
    ],
)

serverless = QiskitServerless()
serverless.upload(template)
print(f"Uploaded program: {title}")

Uploaded program: hamsim_template


In [4]:
serverless.list()

[QiskitFunction(hamsim_template),
 QiskitFunction(function_template_hamsim),
 QiskitFunction(template_hamiltonian_simulation),
 QiskitFunction(aqc_tutorial_L50)]

### Load the template

In [5]:
template = serverless.load(title)

### Prepare hamiltonian and observable

In [6]:
from qiskit.quantum_info import SparsePauliOp

hamiltonian = SparsePauliOp.from_sparse_list(
    [("XX", (i, i + 1), 1.0) for i in range(3)], num_qubits=4
) + SparsePauliOp.from_sparse_list(
    [("YY", (i, i + 1), 1.0) for i in range(3)], num_qubits=4
)
observable = SparsePauliOp.from_sparse_list([("ZZ", (1, 2), 1.0)], num_qubits=4)

### Run the template

In [7]:
job = template.run(
    dry_run=True,
    hamiltonian=hamiltonian,
    observable=observable,
    backend_name="ibm_fez",
    estimator_options={},
    aqc_evolution_time=0.2,
    aqc_ansatz_num_trotter_steps=1,
    aqc_target_num_trotter_steps=32,
    remainder_evolution_time=0.2,
    remainder_num_trotter_steps=4,
    aqc_max_iterations=300,
)

In [8]:
print(job.job_id)

3f0c98e8-b018-4243-828d-0bde255e09c1


### Check the logs (this will return any info about how AQC is behaving)

In [9]:
job.status()

'QUEUED'

In [10]:
print(job.logs())

No logs yet.


### Get the final result (expectation value)

In [11]:
job.result()

{'num_aqc_parameters': 111,
 'target_bond_dimension': 1,
 'aqc_starting_fidelity': 0.9998469036435305,
 'num_iterations': 3,
 'aqc_final_parameters': [-3.141592650080301,
  0.7854329738547067,
  3.1415830013190655,
  -3.141592648357701,
  0.7854005829948971,
  -4.771585317808193e-05,
  1.0536177958486086e-08,
  2.3561859299448105,
  -1.9466309954997306e-07,
  -3.1415926365909472,
  0.13227782458785936,
  3.1415877274788384,
  0.10058281931584818,
  -0.000210284967971987,
  0.0003725528095780092,
  -1.5708061121542478,
  1.5708061752095683,
  -0.7853632963554884,
  -5.165520177604924e-05,
  2.35619686381606,
  -3.251384635320454e-08,
  0.10113150879839808,
  -0.0008073870800575787,
  -3.793581123311004e-07,
  -1.5707961759324534,
  1.5708694553573799,
  3.141590288828051,
  2.6800637339568085e-07,
  0.25355226093861927,
  3.141592901481447,
  0.09924986002645499,
  3.1828626196469066e-07,
  0.0005826564623252421,
  2.087932624658035e-05,
  1.3249363700428107,
  3.1415977288979477,
  3.0

In [12]:
print(job.logs())

2024-12-18 14:25:53,111	INFO job_manager.py:531 -- Runtime env is setting up.
estimator_options = {
    "resilience": {
        "measure_mitigation": true,
        "zne_mitigation": true,
        "zne": {
            "amplifier": "gate_folding",
            "noise_factors": [
                1,
                2,
                3
            ],
            "extrapolated_noise_factors": [
                0.0,
                0.1,
                0.2,
                0.30000000000000004,
                0.4,
                0.5,
                0.6000000000000001,
                0.7000000000000001,
                0.8,
                0.9,
                1.0,
                1.1,
                1.2000000000000002,
                1.3,
                1.4000000000000001,
                1.5,
                1.6,
                1.7000000000000002,
                1.8,
                1.9000000000000001,
                2.0,
                2.1,
                2.2,
                2.3